In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Function for one-hot encoding
def one_hot_encoding(label_data):
    num_samples = label_data.shape[0]
    num_classes = 10  # Assuming 10 classes (0-9)
    encoded_labels = np.zeros((num_samples, num_classes), dtype='int')
    encoded_labels[np.arange(num_samples), label_data] = 1
    return encoded_labels


# Function to read pixel data
def read_pixels(data_path):
    with open(data_path, 'rb') as f:
        raw_data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
    flattened_pixels = raw_data.reshape(-1, 784).astype('float32')
    return flattened_pixels / 255.0


# Function to read label data
def read_labels(data_path):
    with open(data_path, 'rb') as f:
        raw_labels = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    return one_hot_encoding(raw_labels)


# Function to load the MNIST dataset
def load_mnist_data(path):
    X_train = read_pixels(os.path.join(path, "train-images-idx3-ubyte"))
    y_train = read_labels(os.path.join(path, "train-labels-idx1-ubyte"))
    X_test = read_pixels(os.path.join(path, "t10k-images-idx3-ubyte"))
    y_test = read_labels(os.path.join(path, "t10k-labels-idx1-ubyte"))
    return X_train, y_train, X_test, y_test


# Logistic Regression Model
class LogisticRegression:
    def __init__(self, learning_rate=1e-3, reg_lambda=1e-4, epochs=50, batch_size=200, input_dim=784, output_dim=10):
        self.learning_rate = learning_rate
        self.reg_lambda = reg_lambda
        self.epochs = epochs
        self.batch_size = batch_size
        self.weights = np.random.normal(0, 1, (input_dim, output_dim))
        self.biases = np.zeros((1, output_dim))

    def softmax(self, logits):
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Numerical stability
        return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_true, y_pred):
        return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))  # Avoid log(0)

    def train(self, X_train, y_train, X_val, y_val):
        num_samples = X_train.shape[0]
        for epoch in range(self.epochs):
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X_train, y_train = X_train[indices], y_train[indices]

            for start in range(0, num_samples, self.batch_size):
                X_batch = X_train[start:start + self.batch_size]
                y_batch = y_train[start:start + self.batch_size]

                # Forward pass
                logits = np.dot(X_batch, self.weights) + self.biases
                predictions = self.softmax(logits)

                # Compute gradients
                error = predictions - y_batch
                grad_weights = np.dot(X_batch.T, error) / self.batch_size + self.reg_lambda * self.weights
                grad_biases = np.sum(error, axis=0, keepdims=True) / self.batch_size

                # Update weights and biases
                self.weights -= self.learning_rate * grad_weights
                self.biases -= self.learning_rate * grad_biases

            # Print validation accuracy for each epoch
            val_accuracy = self.evaluate(X_val, y_val)
            print(f"Epoch {epoch + 1}/{self.epochs}, Validation Accuracy: {val_accuracy:.4f}")

    def predict(self, X):
        logits = np.dot(X, self.weights) + self.biases
        return np.argmax(self.softmax(logits), axis=1)

    def evaluate(self, X, y_true):
        y_pred = self.predict(X)
        y_true_labels = np.argmax(y_true, axis=1)
        return np.mean(y_true_labels == y_pred)


# Display sample images
def display_sample_images(X_test, y_test, num_samples=3):
    indices = np.random.choice(len(X_test), num_samples, replace=False)
    for i, idx in enumerate(indices):
        image = X_test[idx].reshape(28, 28)
        label = np.argmax(y_test[idx])
        plt.imshow(image, cmap='gray')
        plt.title(f"Label: {label}")
        plt.axis('off')
        plt.show()


# Plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
# Load data
if __name__ == "__main__":
    dataset_path = "/Users/elifsorguc/Desktop/Bilkent/ML/MachineLearning-PCA-LogReg/data/mnist"

    # Load dataset
    X_train, y_train, X_test, y_test = load_mnist_data(dataset_path)

In [ ]:
# sample
if __name__ == "__main__":
    # Display sample images
    display_sample_images(X_test, y_test, num_samples=3)


In [ ]:
# Train
if __name__ == "__main__":
    # Split validation set from training data
    X_val, y_val = X_train[:10000], y_train[:10000]
    X_train, y_train = X_train[10000:], y_train[10000:]

    # Train model
    model = LogisticRegression()
    model.train(X_train, y_train, X_val, y_val)



In [ ]:

if __name__ == "__main__":

    # Generate confusion matrix
    y_test_pred = model.predict(X_test)
    y_test_true = np.argmax(y_test, axis=1)
    cm = np.zeros((10, 10), dtype=int)
    for true, pred in zip(y_test_true, y_test_pred):
        cm[true, pred] += 1

    # Plot confusion matrix
    class_names = [
        "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
        "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
    ]
    plot_confusion_matrix(cm, class_names)
